# Convert class shape

Script converts all objects of selected classes to bitmaps or to bounding boxes

**Input**:
- Existing Project (e.g. "london_roads")
- Conversion rules (see below "_conversions_" dict)

**Output**:
- New Project with converted objects

## Configuration

Edit the following settings for your own case

In [1]:
import supervisely_lib as sly
import collections
import os
from tqdm import tqdm

In [2]:
team_name = "jupyter_tutorials"
workspace_name = "cookbook"
project_name = "tutorial_project"

dst_project_name = "converted_project"

ClassConversion = collections.namedtuple('ClassConversion', 'dst_name dst_shape save_src_class')

# You can convert shape of any class only to BoundignBox (Rectangle) or to Bitmap 
conversions = {"car": ClassConversion(dst_name="car_bbox", dst_shape=sly.Rectangle, save_src_class=True),
               "dog": ClassConversion(dst_name="dog_bitmap", dst_shape=sly.Bitmap, save_src_class=False)}

# Obtain server address and your api_token from environment variables
# Edit those values if you run this notebook on your own PC
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

## Script setup

Import nessesary packages and initialize Supervisely API to remotely manage your projects

In [3]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [4]:
# Get IDs of team, workspace and project by names

team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    raise RuntimeError("Project {!r} not found".format(project_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))
print("Project: id={}, name={}".format(project.id, project.name))

Team: id=30, name=jupyter_tutorials
Workspace: id=76, name=cookbook
Project: id=898, name=tutorial_project


## Get Project Meta of Source Project

Project Meta contains information about classes and tags# Get source project meta

In [5]:
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)

# Check if all classes exist
for src_name in conversions.keys():
    if meta.get_obj_class(src_name) is None:
        raise RuntimeError("Class {!r} not found in source project {!r}".format(src_name, project.name))

## Create Destination project

In [6]:
# check if destination project name already exists. If yes - generate new free name
if api.project.exists(workspace.id, dst_project_name):
    dst_project_name = api.project.get_free_name(workspace.id, dst_project_name)
    
# create remote project
dst_project = api.project.create(workspace.id, dst_project_name)

print("Destination project: id={}, name={!r}".format(dst_project.id, dst_project.name))

Destination project: id=1323, name='converted_project'


## Construct Destination ProjectMeta

In [7]:
# Remove old classes and create new ones

class_mapping = {}

dst_meta = meta.clone()
for src_name, conversion_config in conversions.items():
    obj_class = dst_meta.get_obj_class(src_name)
    if not conversion_config.save_src_class:
        dst_meta = dst_meta.delete_obj_class(src_name)
    
    new_obj_class = obj_class.clone(
        name=conversion_config.dst_name, geometry_type=conversion_config.dst_shape)
    dst_meta = dst_meta.add_obj_class(new_obj_class)
    
    class_mapping[obj_class.name] = new_obj_class

api.project.update_meta(dst_project.id, dst_meta.to_json())
print(dst_meta)

ProjectMeta:
Object Classes
+------------+-----------+----------------+
|    Name    |   Shape   |     Color      |
+------------+-----------+----------------+
|    bike    | Rectangle | [246, 255, 0]  |
|    car     |  Polygon  | [190, 85, 206] |
|   person   |   Bitmap  |  [0, 255, 18]  |
|  car_bbox  | Rectangle | [190, 85, 206] |
| dog_bitmap |   Bitmap  |  [253, 0, 0]   |
+------------+-----------+----------------+
Image Tags
+-------------+--------------+-----------------------+
|     Name    |  Value type  |    Possible values    |
+-------------+--------------+-----------------------+
| cars_number |  any_number  |          None         |
|     like    |     none     |          None         |
|   situated  | oneof_string | ['inside', 'outside'] |
+-------------+--------------+-----------------------+
Object Tags
+---------------+--------------+-----------------------+
|      Name     |  Value type  |    Possible values    |
+---------------+--------------+----------------------

## Helper function, takes annotation, converts labels and returns new annotation

In [8]:
def convert_classes(ann):
    labels = ann.labels
    new_labels = []
    for label in labels:
        dst_class = class_mapping.get(label.obj_class.name, None)
        
        if dst_class is not None:
            dest_shape = conversions[label.obj_class.name].dst_shape
            
            if dest_shape is sly.Bitmap:
                converted_geometry = sly.geometry_to_bitmap(label.geometry)[0]
            elif dest_shape is sly.Rectangle:
                converted_geometry = label.geometry.to_bbox()
            else:
                raise ValueError('Unsupported destination shape ({}) type!'.format(dest_shape.geometry_name()))
                
            new_label = label.clone(obj_class=dst_class, geometry=converted_geometry)
            new_labels.append(new_label) 
        else:
            new_labels.append(label)  
    return ann.clone(labels=new_labels)

## Iterate over all images, convert and upload to destination project

In [9]:
 for dataset in api.dataset.get_list(project.id):
    
    # generate dataset name in destination project if it exists
    dst_dataset_name = dataset.name
    if api.dataset.exists(dst_project.id, dst_dataset_name):
        dst_dataset_name = api.dataset.get_free_name(dst_project.id, dst_project_name)
    # create new dataset in destination project
    dst_dataset = api.dataset.create(dst_project.id, dst_dataset_name)

    print("Processing: project = {!r}, dataset = {!r}".format(project.name, dataset.name), flush=True)
    # add images and annotations from source dataset to destination dataset
    
    images = api.image.get_list(dataset.id)
    with tqdm(total=len(images), desc="Process annotations") as progress_bar:
        for batch in sly.batched(images):
            image_ids = [image_info.id for image_info in batch]
            image_names = [image_info.name for image_info in batch]
            
            ann_infos = api.annotation.download_batch(dataset.id, image_ids)

            anns_to_upload = []
            for ann_info in ann_infos:
                ann = sly.Annotation.from_json(ann_info.annotation, meta)
                new_ann = convert_classes(ann)
                anns_to_upload.append(new_ann)
            
            dst_image_infos = api.image.upload_ids(dst_dataset.id, image_names, image_ids)
            dst_image_ids = [image_info.id for image_info in dst_image_infos]
            api.annotation.upload_anns(dst_image_ids, anns_to_upload)
            progress_bar.update(len(batch))

Processing: project = 'tutorial_project', dataset = 'dataset_01'


Process annotations: 100%|██████████| 3/3 [00:00<00:00, 11.32it/s]

Processing: project = 'tutorial_project', dataset = 'dataset_02'



Process annotations: 100%|██████████| 2/2 [00:00<00:00, 11.65it/s]


## Done!